In [1]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
panda = Robot(verbose=False)
# data generation
X, y = load_data(robot=panda)
# build dimension reduction model
hnne, ds, loader = get_hnne_model(X, y)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()

hnne load successfully from ./weights/panda/hnne.pickle
Model load successfully from ./weights/panda/nsf.pth


In [5]:
df, err = test_l2_err(config, 
                      robot=panda, 
                      loader=loader, 
                      model=flow, 
                      num_data=100, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

,l2_err,time_diff(per 1000)
count,100.000000,100.000000
mean,0.007130,0.062600
std,0.003279,0.023034
min,0.003543,0.060000
25%,0.005114,0.060000
50%,0.006194,0.060000
75%,0.007670,0.060000
max,0.020976,0.290000


In [6]:
nflow = get_nflow_model(flow=flow)
df, err = test_l2_err(config, 
                      robot=panda, 
                      loader=loader, 
                      model=nflow, 
                      num_data=100, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

,l2_err,time_diff(per 1000)
count,100.000000,100.000000
mean,0.004753,0.062700
std,0.002501,0.018305
min,0.002221,0.060000
25%,0.003361,0.060000
50%,0.004230,0.060000
75%,0.005440,0.060000
max,0.024142,0.240000


In [7]:
traj_dir = sample_ee_traj(robot=panda, load_time='')

100%|██████████| 20/20 [00:00<00:00, 38251.75it/s]

mkdir ./data/panda/trajectory/05251543
./data/panda/trajectory/05251543/ load successfully.


In [8]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=nflow, robot=panda, traj_dir=traj_dir)

[[0.76221121 0.27924557 0.83192703 0.28457488]
 [0.71377316 0.26820641 0.80690692 0.27729773]
 [0.7307427  0.33987673 0.87146256 0.25010577]]
          l2_err   log_prob
count  20.000000  20.000000
mean    0.007993 -14.597330
std     0.009960   1.576296
min     0.001536 -16.112041
25%     0.002456 -15.426632
50%     0.004100 -15.261176
75%     0.005885 -14.518756
max     0.035703  -9.935677
          l2_err   log_prob
count  20.000000  20.000000
mean    0.005665 -14.874350
std     0.005969   1.716685
min     0.001138 -16.640348
25%     0.002923 -15.826204
50%     0.003729 -15.479504
75%     0.005544 -14.775248
max     0.027604  -9.923030
          l2_err   log_prob
count  20.000000  20.000000
mean    0.005263 -14.791424
std     0.005021   1.614270
min     0.001000 -16.192394
25%     0.002333 -15.809322
50%     0.004656 -15.047681
75%     0.005901 -14.554054
max     0.024531  -9.193552
(array([], dtype=int64),)
[]
0.15986617295315575
(array([], dtype=int64),)
[]
0.11329430243102129
(arr

In [11]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=nflow, robot=panda, traj_dir=traj_dir)

(array([], dtype=int64),)
[]
0.15986617295315575
(array([], dtype=int64),)
[]
0.11329430243102129
(array([], dtype=int64),)
[]
0.10526919017708636
